In [ ]:
import os
import pandas as pd
import numpy as np
import json
from unidecode import unidecode

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
tables_folder = '<folder_with_generated_tables>'
output_folder = '<folder_to_save_results>'

In [ ]:
def normalize_value(value, is_date=False):
    if value != value:
        return ''

    if is_date:
        try:
            return date_parser.parse(value)
        except:
            try:
                return pd.to_datetime(value)
            except:
                pass    
        
    if type(value).__module__ == 'numpy':
        value = value.item()

    if isinstance(value, str):            
        if value.startswith('-') and value.replace('-', '').replace(',', '').isdigit():                
            return int(value.replace(',', ''))
        if value.replace(',', '').isdigit():                
            return int(value.replace(',', ''))
        if value.startswith('=') and value.isdigit():                
            return int(value.replace('=', ''))
        if value.startswith('-') and value.replace('-', '').replace(',', '').replace('.', '').isdigit():                
            return float(value.replace(',', ''))
        if value.replace(',', '').replace('.', '').isdigit():                
            return float(value.replace(',', ''))
        
        value = value.strip().lower()

        if value in ('none', 'n/a', 'nan', '-'):
            return '' 

        value = value.replace('&', 'and')

        if value == 'united states':
            return 'usa'
        if value == 'united kingdom':
            return 'uk'
        
        value = unidecode(value)        
        value = ''.join(c for c in value if c.isalnum()) 
        return value

    return value

def normalize_key(value, is_date=False):
    if value != value:
        return ''

    if is_date:
        try:
            return str(date_parser.parse(value))
        except:
            try:
                return str(pd.to_datetime(value))
            except:
                pass  

    if isinstance(value, str):  
        value = value.strip().lower()

        if value in ('none', 'n/a', 'nan', '-', '--', 'unknown'):
            return '' 

        value = value.replace('&', 'and')

        if value == 'united states':
            return 'usa'
        if value == 'united kingdom':
            return 'uk'

        value = unidecode(value)        
        value = ''.join(c for c in value if c.isalnum()) 
        return value

    return str(value)

def normalize_primary_columns(df, norm_columns, date_columns, primary_columns, keys_type):
    for col in norm_columns:
        df[col] = df[col].apply(normalize_key, col in date_columns)
    
    for col, key_type in zip(primary_columns, keys_type):
        if key_type == 'year':
            df[col] = df[col].astype(float).astype(int)
            
        df[col] = df[col].astype(str)
        
    return [tuple(r) for r in df[primary_columns].to_numpy()]    

def find_row(df, columns, values):
    query = ' & '.join([f'(`{col}`=="{value}")' for col, value in zip(columns, values)])    
    return df.query(query)                    

def evaluate_table(df_fetched, df_ref, primary_columns, keys_type, date_columns, epsilons):
    columns = df_ref.columns
    df_fetched.columns = columns    
    df_fetched = df_fetched.drop_duplicates(subset=primary_columns)   

    norm_columns = set(primary_columns)        
    for pc in primary_columns:
        df_fetched = df_fetched[df_fetched[pc].notna()]    
    
    fetched_entities = normalize_primary_columns(df_fetched, norm_columns, date_columns, primary_columns, keys_type)
    ref_entities = normalize_primary_columns(df_ref, norm_columns, date_columns, primary_columns, keys_type)

    total_matches = 0
    key_matches = 0

    for fetched_entity in fetched_entities:
        if fetched_entity in ref_entities: 
            row_fetched = find_row(df_fetched, primary_columns, fetched_entity)
            row_ref = find_row(df_ref, primary_columns, fetched_entity)
            key_matches += 1
            
            for column in columns:
                try:
                    value_fetched = row_fetched[column].values[0]
                    value_ref = row_ref[column].values[0]

                    norm_value_fetched = normalize_value(value_fetched, column in date_columns)
                    norm_value_ref = normalize_value(value_ref, column in date_columns)

                    if norm_value_fetched == norm_value_ref:    
                        total_matches += 1
                    elif column in epsilons and norm_value_ref != '' and norm_value_fetched != '':
                        if norm_value_ref * 0.999 < norm_value_fetched < norm_value_ref * 1.001:
                            total_matches += 1   
                except:
                    pass
            
    recall = total_matches/(df_ref.shape[0] * df_ref.shape[1])
    precision = total_matches/(df_fetched.shape[0] * df_fetched.shape[1])
    f1_score = 2*recall*precision/(recall+precision) if (recall + precision) > 0 else 0.0

    keys_recall = key_matches/len(ref_entities)
    keys_precision = key_matches/len(fetched_entities)
    keys_f1_score = 2*keys_recall*keys_precision/(keys_recall+keys_precision) if (keys_recall + keys_precision) > 0 else 0.0
    
    nk = len(primary_columns)
    
    non_keys_recall = (total_matches - key_matches*nk) / (df_ref.shape[0] * (df_ref.shape[1] - nk))
    non_keys_precision = (total_matches - key_matches*nk) / (df_fetched.shape[0] * (df_fetched.shape[1] - nk))
    non_keys_f1_score = 2*non_keys_recall*non_keys_precision/(non_keys_recall+non_keys_precision) if (non_keys_recall + non_keys_precision) > 0 else 0.0
     
    relative_non_key_accuracy = (total_matches - key_matches*nk) / (key_matches * (df_ref.shape[1] - nk))    
        
    return keys_recall, keys_precision, keys_f1_score, non_keys_recall, non_keys_precision, non_keys_f1_score, recall, precision, f1_score, relative_non_key_accuracy

## JSON

In [ ]:
metadata_path="../benchmark/cfg.json"

with open(metadata_path, "rb") as f:
    metadata = json.load(f)

In [ ]:
tables = []

keys_recall_scores = []
keys_precision_scores = []
keys_f1_scores = []
non_keys_recall_scores = []
non_keys_precision_scores = []
non_keys_f1_scores = []
recall_scores = []
precision_scores = []
f1_scores = []
rel_nk_acc_scores = []

for i in range(100):
    idx = "%d" % i
    md = metadata[idx]
    print(md['name'])
    try:
        fetched_table = pd.read_csv(os.path.join(tables_folder, "%s.csv" % md['name']))
        gt_table = pd.read_csv(md['path'])
        primary_columns = md['keys']
        keys_type = md['keys_type']
        date_columns = md['dateColumns']
        epsilons = md['epsilons']
        kr, kp, kf1, nkr, nkp, nkf1, r, p, f1, rnka  = evaluate_table(fetched_table, 
                                                                       gt_table, 
                                                                       primary_columns, 
                                                                       keys_type,
                                                                       date_columns, 
                                                                       epsilons)
    except:    
        kr, kp, kf1, nkr, nkp, nkf1, r, p, f1, rnka = 0, 0, 0, 0, 0, 0, 0, 0, 0, 0
        
    tables.append(md['name'])
    
    keys_recall_scores.append(kr)
    keys_precision_scores.append(kp)
    keys_f1_scores.append(kf1)
    non_keys_recall_scores.append(nkr)
    non_keys_precision_scores.append(nkp)
    non_keys_f1_scores.append(nkf1)
    recall_scores.append(r)
    precision_scores.append(p)
    f1_scores.append(f1)
    rel_nk_acc_scores.append(rnka)
    
    print("====================")
    print("Keys Recall: %.4f" % kr)
    print("Keys Precision: %.4f" % kp)
    print("Keys F1: %.4f" % kf1)
    
    print("Non Keys Recall: %.4f" % nkr)
    print("Non Keys Precision: %.4f" % nkp)
    print("Non Keys F1: %.4f" % nkf1)
    
    print("Relative Non Keys Accuracy: %.4f" % rnka)
    
    print("Recall: %.4f" % r)
    print("Precision: %.4f" % p)
    print("F1: %.4f" % f1)    
    
    print("====================")    
    
res_df = pd.DataFrame([tables, 
                       keys_recall_scores,
                       keys_precision_scores,
                       keys_f1_scores,
                       non_keys_recall_scores,
                       non_keys_precision_scores,
                       non_keys_f1_scores,
                       rel_nk_acc_scores,
                       recall_scores,
                       precision_scores, 
                       f1_scores                       
                       ]).T
res_df.columns = ['Table', 
                  'Keys_Recall', 'Keys_Precision', 'Keys_F1_Score',
                  'Non_Keys_Recall', 'Non_Keys_Precision', 'Non_Keys_F1_Score', 'Rel_Non_Keys_Accuracy',
                  'Recall', 'Precision', 'F1_Score'
                  ]

res_df['Keys_Recall'] = res_df['Keys_Recall'].astype(float).round(4)
res_df['Keys_Precision'] = res_df['Keys_Precision'].astype(float).round(4)
res_df['Keys_F1_Score'] = res_df['Keys_F1_Score'].astype(float).round(4)
res_df['Non_Keys_Recall'] = res_df['Non_Keys_Recall'].astype(float).round(4)
res_df['Non_Keys_Precision'] = res_df['Non_Keys_Precision'].astype(float).round(4)
res_df['Non_Keys_F1_Score'] = res_df['Non_Keys_F1_Score'].astype(float).round(4)
res_df['Rel_Non_Keys_Accuracy'] = res_df['Rel_Non_Keys_Accuracy'].astype(float).round(4)
res_df['Recall'] = res_df['Recall'].astype(float).round(4)
res_df['Precision'] = res_df['Precision'].astype(float).round(4)
res_df['F1_Score'] = res_df['F1_Score'].astype(float).round(4)

means = pd.DataFrame(['All'] + res_df.mean(axis=0, numeric_only=True).tolist()).T
means.columns = res_df.columns        
res_df = pd.concat([res_df, means], axis=0)

res_df.to_csv("%s/scores.csv" % result_folder, index=False)